<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/course_project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT Project

- Student Name: Aaron Leino
- Date: 6.5.2025
- Chosen Corpus: Stanford Sentiment Treebank (SST-2)

### Corpus information

- Description of the chosen corpus: The Stanford Sentiment Treebank contains movie review sentences from Rotten Tomatoes. The reviews are annotated with binary sentiment labels: positive and negative.
- Paper(s) and other published materials related to the corpus: Socher et al., 2013: Recursive Deep Models for Semantic Compositionality Over a Sentiment Treebank
- State-of-the-art performance (best published results) on this corpus: T5-11B and MT-DNN-SMART, both with 97.5% accuracy. (Papers With Code)

---

## 1. Setup

In [15]:
!pip install -q datasets transformers evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch
from sklearn.model_selection import ParameterGrid
import optuna

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [10]:
dataset = load_dataset("glue", "sst2")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


### 2.2. Preprocessing

In [17]:
# Use DistilBERT as tokenizer

distilbert = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(distilbert)

# Tokenize sentences
def tokenize_func(inputdata):
    return tokenizer(inputdata["sentence"], truncation = True, padding = "max_length", max_length=128)
tokenized_dataset = dataset.map(tokenize_func, batched=True)

# The test set does not have labels so I split the train data.
split1 = tokenized_dataset["train"].train_test_split(test_size=200, seed=2025)
test_dataset = split1["test"]  
remaining_train = split1["train"] 
train_dataset = remaining_train.select(range(1000))

validation_dataset = tokenized_dataset["validation"].select(range(200))


---

## 3. Machine learning model

### 3.1. Model training

In [18]:

# Define accuracy metrics
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

### 3.2 Hyperparameter optimization

In [19]:
def model_init(): #for initializing a fresh model each time
    return AutoModelForSequenceClassification.from_pretrained(distilbert, num_labels=2)

# use optuna for parameter optimization
# define parameter space for optuna 

def optuna_hp_space(ex):
    return {
      "learning_rate": ex.suggest_categorical("learning_rate", [2e-5, 3e-5, 5e-5]),
      "num_train_epochs": ex.suggest_int("num_train_epochs", 2, 3),
      "per_device_train_batch_size": ex.suggest_categorical("per_device_train_batch_size", [8, 16]),
      "weight_decay": ex.suggest_loguniform("weight_decay", 1e-6, 1e-1),
    }

#define the base arguments for optuna
base_args = TrainingArguments(
    output_dir="./optuna_results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",               
    load_best_model_at_end=False,     
    disable_tqdm=True
)

# hyperparameter search
trainer = Trainer(
    model_init=model_init,
    args=base_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=12,
    hp_space=optuna_hp_space,
)


print("Best hyperparameters:", best_run.hyperparameters)



c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-05-07 11:14:41,290] A new study created in memory with name: no-name-7eef530c-c6c9-4846-b96d-9eaf835bb3bc
C:\Users\leino\AppData\Local\Temp\ipykernel_18212\2038782950.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/release

{'loss': 0.5075, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
{'eval_loss': 0.4407897889614105, 'eval_accuracy': 0.8, 'eval_runtime': 5.5042, 'eval_samples_per_second': 36.336, 'eval_steps_per_second': 4.542, 'epoch': 1.0}


c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.1864, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}
{'eval_loss': 0.6523927450180054, 'eval_accuracy': 0.79, 'eval_runtime': 4.7747, 'eval_samples_per_second': 41.887, 'eval_steps_per_second': 5.236, 'epoch': 2.0}


c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

{'eval_loss': 0.4480636715888977, 'eval_accuracy': 0.8, 'eval_runtime': 13.4333, 'eval_samples_per_second': 14.888, 'eval_steps_per_second': 1.861, 'epoch': 1.0}


C:\Users\leino\AppData\Local\Temp\ipykernel_18212\2038782950.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": ex.suggest_loguniform("weight_decay", 1e-6, 1e-1),
c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

{'loss': 0.5078, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


[I 2025-05-07 12:05:05,203] Trial 11 pruned. 


{'eval_loss': 0.4426151216030121, 'eval_accuracy': 0.8, 'eval_runtime': 13.4545, 'eval_samples_per_second': 14.865, 'eval_steps_per_second': 1.858, 'epoch': 1.0}
Best hyperparameters: {'learning_rate': 5e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 8, 'weight_decay': 0.002314276187863276}


### 3.3. Evaluation on test set

In [20]:

# define the best model by the previous optimization
# Extract the best parameters
hp = best_run.hyperparameters
best_args = TrainingArguments(
    output_dir="./best_model",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=hp["learning_rate"],
    per_device_train_batch_size=hp["per_device_train_batch_size"],
    num_train_epochs=hp["num_train_epochs"],
    weight_decay=hp.get("weight_decay", 0.0),
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    disable_tqdm=True,
)

# Combine validation and train data for larger training data
from datasets import concatenate_datasets
full_train = concatenate_datasets([train_dataset, validation_dataset])

best_trainer = Trainer(
    model_init(),
    args= best_args,
    train_dataset= full_train,
    eval_dataset=full_train,
    compute_metrics=compute_metrics,
)

#train the model
best_trainer.train()

# calculate results on test data
test_results = best_trainer.evaluate(test_dataset)
print("Test accuracy:", test_results["eval_accuracy"])

c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.4702, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
{'eval_loss': 0.15192928910255432, 'eval_accuracy': 0.9525, 'eval_runtime': 30.2194, 'eval_samples_per_second': 39.71, 'eval_steps_per_second': 4.964, 'epoch': 1.0}


c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.1774, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}
{'eval_loss': 0.035744521766901016, 'eval_accuracy': 0.9925, 'eval_runtime': 26.4285, 'eval_samples_per_second': 45.406, 'eval_steps_per_second': 5.676, 'epoch': 2.0}


c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.0567, 'learning_rate': 0.0, 'epoch': 3.0}
{'eval_loss': 0.018343310803174973, 'eval_accuracy': 0.9958333333333333, 'eval_runtime': 26.626, 'eval_samples_per_second': 45.069, 'eval_steps_per_second': 5.634, 'epoch': 3.0}
{'train_runtime': 464.9755, 'train_samples_per_second': 7.742, 'train_steps_per_second': 0.968, 'train_loss': 0.2347829967074924, 'epoch': 3.0}


c:\Users\leino\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.7507790923118591, 'eval_accuracy': 0.835, 'eval_runtime': 4.3702, 'eval_samples_per_second': 45.765, 'eval_steps_per_second': 5.721, 'epoch': 3.0}
Test accuracy: 0.835


---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [7]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [8]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [9]:
# Include your annotated out-of-domain data here